![Title](img/pycon.png)


## Geospatial Data Processing, Analysis and Visualization using Python

Prof. Martin Christen<br/>
mailto:martin.christen@fhnw.ch<br/>
Twitter: @MartinChristen<br/>
LinkedIn: https://www.linkedin.com/in/martinchristen/



### This notebook can be found at: https://github.com/martinchristen/PyConBy2019



In this notebook the following will be covered:

**Vector File Formats** (Shapefiles, GeoJSON, KML, GeoPackage, ...)<br/>
**Raster File Formats** (GeoTiff, JPG, ...)

* Shapely: Manipulation and analysis of geometric objects ( https://github.com/Toblerity/Shapely )
* Fiona - The pythonic way to handle vector data ( https://github.com/Toblerity/Fiona )
* rasterio - The pythonic way to handle raster data ( https://github.com/mapbox/rasterio )
* pyproj - transforming spatial reference systems ( https://github.com/jswhit/pyproj ) 
* Geospatial analysis using GeoPandas ( https://github.com/geopandas/geopandas ) 
* Creating maps using Folium ( https://github.com/python-visualization/folium ) and/or ipyleaflet ( https://github.com/jupyter-widgets/ipyleaflet )
  



### Data used in this notebook

* Natural Earth Dataset, https://www.naturalearthdata.com/ (public domain)
* Blue Marble: Next Generation was produced by Reto Stöckli, NASA Earth Observatory (NASA Goddard Space Flight Center
* geonames: http://www.geonames.org/ CC License
                                                                                     
To keep it simple, I put it on the git repository

![Title](img/overview_tools.png)

## 1. Installation

(This tutorial requires anaconda, if you don't have it yet, download it here: https://www.anaconda.com/download/ )

**This notebook requires Python 3.6 (or higher)** (jupyter Notebook or Jupyter Lab using Chrome)

### Installing Modules (conda)

Install the main modules using conda, dependencies will be resolved (gdal etc.)

    conda install shapely
    conda install fiona
    conda install rasterio
    conda install geopandas
    conda install folium -c conda-forge
    conda install ipyleaflet -c conda-forge 

## 2. Shapely

Shapely is a BSD-licensed Python package for **manipulation** and **analysis** of **planar geometric objects**. 

* Shapely is **not** concerned with data formats or coordinate systems.
* Shapely is based on the widely deployed GEOS (the engine of PostGIS) and JTS (from which GEOS is ported) libraries.


### Simple Feature Access

**Simple Feature Access** is both an Open Geospatial Consortium (OGC) and International Organization for Standardization (ISO) standard **ISO 19125** that specifies a common storage and access model of mostly two-dimensional geometries (point, line, polygon, multi-point, multi-line, etc.) used by geographic information systems.





Shapely supports the following **Simple Feature Access** Features:

* Point
* LineString
* LinearRing        
* Polygon
* MultiLineString
* MultiPoint
* MultiPolygon

![Features](img/Features.png)

### Examples

In [ ]:
from shapely.geometry import Polygon

polygon1 = Polygon([(30, 10), (40, 40), (20, 35), (10, 20), (30, 10)])

print(f"Polygon area: {polygon1.area}, polygon length: {polygon1.length}") 

You can display shapely objects directly in the Jupyter Notebook (usually for "debug" purposes)

It is also possible to display this in matplotlib/descartes, we will see later how to bring this directly on a map)

In [ ]:
polygon1

In [ ]:
polygon2 = Polygon([(20,20),(80,30),(50,40),(20,20)])
polygon2

In [ ]:
polygon2.union(polygon1)

In [ ]:
polygon2.intersection(polygon1)

In [ ]:
polygon2.symmetric_difference(polygon1)

In [ ]:
result = polygon2.symmetric_difference(polygon1)

print(f"Polygon area: {result.area}, polygon length: {result.length}")

### Well Known Text (WKT)

In [ ]:
result.wkt

In [ ]:
s = result.wkt
type(s)

In [ ]:
import shapely.wkt

mypolygon = shapely.wkt.loads(s)
mypolygon

### Binary operations on shapes:

- **contains** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **intersects** (Returns True if the boundary and interior of the object intersect in any way with those of the other.)
- **witin** (Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).
- **touches** (Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.)
- **crosses** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **equals** (Returns True if the set-theoretic boundary, interior, and exterior of the object coincide with those of the other.)

In [ ]:
polygon1.intersects(polygon2)

In [ ]:
polygon1.within(polygon2)

In [ ]:
polygon1.equals(polygon1)

## 3. Fiona

Spatial information is not only geometry. Together with it we always have metadata (properties). Lets look at a first dataset.


### Reading Vector Data

In [ ]:
import fiona

c = fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r')

airport = next(iter(c))
airport

In [ ]:
airport['properties']['name']

In [ ]:
airport['geometry']['type']

In [ ]:
airport['geometry']['coordinates']

In [ ]:
c.close()

### Two ways to read vector data

Lets read all data. There are basically two ways:

a) load everything to memory: (if dataset isn't too large...)

    alldata = list(c)
    
b) iterate through all data: (one by one):

    for element in c:
        ...
    

In [ ]:
with fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r') as c:
    for airport in c:
        if airport['properties']['iata_code'] == "MSQ":
            print(airport['properties']['name'])
            print(airport['geometry']['coordinates'])
            print(airport['properties']['wikipedia'])        


Coordinate System ? (Coordinate Reference System?)

In [ ]:
with fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r') as c:
    print(c.crs)

EPSG: 4326 look it up at: http://epsg.io/

( http://epsg.io/4326 )


For transformations to other spatial reference systems, use **pyproj**


### Lets look at another dataset:

In [ ]:
import fiona

c = fiona.open('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp', 'r', encoding="utf-8")

country = next(iter(c))
#country

In [ ]:
print(country['properties']['NAME'])

In [ ]:
print(country['properties']['NAME_ZH'])

In [ ]:
print(country['properties']['NAME_RU'])

In [ ]:
print(country['properties']['CONTINENT'])

In [ ]:
print(country['properties']['POP_EST'])
print(country['properties']['POP_YEAR'])

### Example: Find the "Belarus" Polygon and store it

In [ ]:
with fiona.open('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp', 'r') as c:
    for country in c:
        if country['properties']['NAME'] == "Belarus":
            print(country['properties']['POP_EST'])
            print(country['properties']['POP_YEAR'])   
            print(country['geometry']['type'])
            # print(country['geometry']['coordinates']) # you don't want to print all coordinates!
            geometry = country['geometry']


### Example: find Airports inside the Belarus polygon

In [ ]:
from shapely.geometry import shape
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon

by = Polygon(shape(geometry))
by

In [ ]:
from shapely.geometry import Point

with fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r') as c:
    for airport in c:      
        position = Point(airport['geometry']['coordinates'])
        if position.within(by):
            print(airport['properties']['iata_code'], airport['properties']['name'], airport['geometry']['coordinates'])

## 4. Rasterio

### Reading data

In [ ]:
import rasterio

dataset = rasterio.open('data/BlueMarble.tif', 'r')

In [ ]:
dataset.name

In [ ]:
dataset.mode

In [ ]:
dataset.count # number of raster bands, in our case 3 for r,g,b

In [ ]:
dataset.indexes

In [ ]:
dataset.width, dataset.height

In [ ]:
dataset.crs

In [ ]:
dataset.bounds

In [ ]:
dataset.transform  # affine transformation pixel to crs

In [ ]:
dataset.transform * (0, 0)

In [ ]:
~dataset.transform # inverse affine transformation crs to pixel

In [ ]:
~dataset.transform * (0,0)

In [ ]:
px,py = ~dataset.transform * (27.487117, 53.869146) # Minsk coordinates to pixel coordinates! 
print(px,py)

In [ ]:
print(dataset.bounds.left)
print(dataset.bounds.bottom)
print(dataset.bounds.right)
print(dataset.bounds.top)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

In [ ]:
rgb = np.dstack((r,g,b))  # stack r,g,b so we can display it...
rgb

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))
ax.imshow(rgb, interpolation='nearest')
ax.plot(px,py, 'ro'); # IBB Minsk

## 5. Geopandas

GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas **extends** the datatypes used by pandas to **allow spatial operations** on **geometric types**. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting.
(geopandas.org)


In [ ]:
import pandas as pd

df = pd.read_csv('data/cities5k.csv', encoding="utf-8", sep=",", header=None, low_memory=False)
df.head()

In [ ]:
df2 = df[[1,4,5,14]]
df2.columns = ["name", "lat", "lng", "population"]
df2.head()

In [ ]:
df2.query("name == 'Minsk'")

### Geometry Column

Now lets create a geopandas dataframe. Basically just create a new column "geometry" - using Shapely to create Points

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

In [ ]:
gdf.head()

In [ ]:
gdf = gdf.drop(['lat', 'lng'], axis=1) # remove redundant data, we only want "geometry"

In [ ]:
gdf.plot(color='green', markersize=1, figsize=(15,9))

In [ ]:
# Export to Shapefile
# gdf.to_file("cities.shp", driver="Shapefile", encoding="utf-8")

In [ ]:
cities = gdf
cities.head()  # store as "cities" for later

In [ ]:
# another query:

bigcities = cities[cities.population > 10_000_000]

bigcities = bigcities.sort_values(['population'], ascending=False)
bigcities

### Dataset with Polygons

Now we load the admin 0 country files again:

In [ ]:
countries = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
countries.head()

In [ ]:
countries2 = countries[["NAME", "POP_EST", "geometry"]]
countries2.head()

In [ ]:
by = countries2[countries2['NAME'] == "Belarus"]
by

In [ ]:
by.plot(figsize=(15,9))

In [ ]:
restoftheworld = countries2[countries2['NAME'] != "Belarus"]

ax = by.plot(figsize=(15,9), color="green")
restoftheworld.plot(ax=ax, color="black");

In [ ]:
#type(de.geometry) # GeoSeries
type(by.iloc[0].geometry) # shapely.geometry.multipolygon.MultiPolygon or Polygon

### Query positions within Polygons

In [ ]:
# Get All Cities within Belarus Polygon

bycities = cities[cities.within(by.iloc[0].geometry)]
bycities.head(10)

## 6. Folium

In [ ]:
import folium


m = folium.Map(location=[49.001575, 8.3832086], zoom_start=17)
m

### GeoPandas & Folium

our previous polygon "by" can be converted to GeoJson

In [ ]:
import folium

center = [49.001575, 8.3832086] # ZKM in Karlsruhe!
map_zkm = folium.Map(center, zoom_start=5)   

folium.GeoJson(by).add_to(map_zkm)

map_zkm

In [ ]:
import folium

center = [49.001575, 8.3832086] # ZKM in Karlsruhe!
map_zkm = folium.Map(center, zoom_start=5)   

folium.GeoJson(by,style_function=lambda feature: {
        'fillColor': 'green',   # you can also replace this with functions with feature as argument
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(map_zkm)

map_zkm

In [ ]:
# remember bigcities? Let's display them on a map using markers

bigcities.head()

In [ ]:
world_map = folium.Map(location=[0,0], zoom_start=2)

def f(row):
    lng, lat = row[2].x, row[2].y
    name = row[0]
    population = row[1]
    folium.Marker([lat, lng], 
              popup=name + "<br/>" + str(int(population)),
              icon=folium.Icon(color="red", prefix="fa", icon="home")).add_to(world_map)

bigcities.apply(f, axis=1);
    
world_map



In [ ]:
world_map.save("mymap.html")

# Another Example: Geodetic Line

In [ ]:
center = [53.869146, 27.487117] 
zoom = 4

In [ ]:
# Geodetic Line from Minsk to New York
import pyproj
g = pyproj.Geod(ellps='WGS84')

startlong = 27.487117
startlat = 53.869146
endlong = -74.001457
endlat = 40.7094328

lonlats = g.npts(startlong, startlat, endlong, endlat, 14)
lonlats = [(startlong, startlat)] + lonlats +  [(endlong, endlat)]    # Add Start and End

In [ ]:
world_map = folium.Map(location=center, zoom_start=zoom)

for pos in lonlats:
    folium.Marker([pos[1], pos[0]], 
                  icon=folium.Icon(color="red", prefix="fa", icon="airport")).add_to(world_map)

    
world_map

## More Geo?

![gp2019](img/geopython.png)

## Don't miss EP2019!

![ep2019](img/ep2019.png)

### Questions


Download this Notebook here:  https://github.com/martinchristen/PyConBy2019


mailto:martin.christen@fhnw.ch<br/>
Twitter: @MartinChristen<br/>
LinkedIn: https://www.linkedin.com/in/martinchristen/